In [16]:
import xarray as xr
import zarr
import numcodecs
import constants
import numpy as np
import glob
import torch

In [17]:
dataset_path = "data/era5_uk_reduced/samples/train"
nc_files = glob.glob(f'{constants.DATASET_PATH}/global/*.nc')
print(nc_files)

['/work/ec249/ec249/bet20/dataset/era5/global/2022_02.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_12.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_06.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_08.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_04.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_01.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_03.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_07.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_09.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_10.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_11.nc', '/work/ec249/ec249/bet20/dataset/era5/global/2022_05.nc']


In [18]:
data = xr.open_dataset(nc_files[0])

In [49]:
data

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, level: 7, time: 112)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 50 150 250 400 600 850 1000
  * time       (time) datetime64[ns] 2022-02-01 ... 2022-02-28T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 ...
    q          (time, level, latitude, longitude) float32 ...
    t          (time, level, latitude, longitude) float32 ...
    u          (time, level, latitude, longitude) float32 ...
    v          (time, level, latitude, longitude) float32 ...
    w          (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-01-02 23:49:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [63]:
from era5_data_proc import uk_subset
uk_data = uk_subset(data)
uk_data

<xarray.Dataset>
Dimensions:    (longitude: 57, latitude: 65, level: 7, time: 112)
Coordinates:
  * longitude  (longitude) float32 350.0 350.2 350.5 350.8 ... 3.25 3.5 3.75 4.0
  * latitude   (latitude) float32 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 50 150 250 400 600 850 1000
  * time       (time) datetime64[ns] 2022-02-01 ... 2022-02-28T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 1.915e+05 ... 2.17e+03
    q          (time, level, latitude, longitude) float32 3.164e-06 ... 0.00446
    t          (time, level, latitude, longitude) float32 191.8 192.0 ... 283.2
    u          (time, level, latitude, longitude) float32 58.46 58.45 ... -1.911
    v          (time, level, latitude, longitude) float32 -13.31 ... -0.5006
    w          (time, level, latitude, longitude) float32 0.06605 ... 0.1339
Attributes:
    Conventions:  CF-1.6
    history:      2024-01-02 23:49:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [59]:
uk_data['longitude'] = xr.where(uk_data['longitude'] > 180, uk_data['longitude'] - 360, uk_data['longitude'])

In [60]:
uk_data

<xarray.Dataset>
Dimensions:    (longitude: 57, latitude: 65, level: 7, time: 112)
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.75 -9.5 -9.25 ... 3.25 3.5 3.75 4.0
  * latitude   (latitude) float32 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 50 150 250 400 600 850 1000
  * time       (time) datetime64[ns] 2022-02-01 ... 2022-02-28T18:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 1.915e+05 ... 2.17e+03
    q          (time, level, latitude, longitude) float32 3.164e-06 ... 0.00446
    t          (time, level, latitude, longitude) float32 191.8 192.0 ... 283.2
    u          (time, level, latitude, longitude) float32 58.46 58.45 ... -1.911
    v          (time, level, latitude, longitude) float32 -13.31 ... -0.5006
    w          (time, level, latitude, longitude) float32 0.06605 ... 0.1339
Attributes:
    Conventions:  CF-1.6
    history:      2024-01-02 23:49:10 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [52]:
uk_data.isel(time=0).to_array().values.shape

(6, 7, 65, 57)

In [58]:
latitude = uk_data.latitude.values
latitude

array([63.  , 62.75, 62.5 , 62.25, 62.  , 61.75, 61.5 , 61.25, 61.  ,
       60.75, 60.5 , 60.25, 60.  , 59.75, 59.5 , 59.25, 59.  , 58.75,
       58.5 , 58.25, 58.  , 57.75, 57.5 , 57.25, 57.  , 56.75, 56.5 ,
       56.25, 56.  , 55.75, 55.5 , 55.25, 55.  , 54.75, 54.5 , 54.25,
       54.  , 53.75, 53.5 , 53.25, 53.  , 52.75, 52.5 , 52.25, 52.  ,
       51.75, 51.5 , 51.25, 51.  , 50.75, 50.5 , 50.25, 50.  , 49.75,
       49.5 , 49.25, 49.  , 48.75, 48.5 , 48.25, 48.  , 47.75, 47.5 ,
       47.25, 47.  ], dtype=float32)

In [57]:
longitudes = uk_data.longitude.values
longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
longitudes

array([-10.  ,  -9.75,  -9.5 ,  -9.25,  -9.  ,  -8.75,  -8.5 ,  -8.25,
        -8.  ,  -7.75,  -7.5 ,  -7.25,  -7.  ,  -6.75,  -6.5 ,  -6.25,
        -6.  ,  -5.75,  -5.5 ,  -5.25,  -5.  ,  -4.75,  -4.5 ,  -4.25,
        -4.  ,  -3.75,  -3.5 ,  -3.25,  -3.  ,  -2.75,  -2.5 ,  -2.25,
        -2.  ,  -1.75,  -1.5 ,  -1.25,  -1.  ,  -0.75,  -0.5 ,  -0.25,
         0.  ,   0.25,   0.5 ,   0.75,   1.  ,   1.25,   1.5 ,   1.75,
         2.  ,   2.25,   2.5 ,   2.75,   3.  ,   3.25,   3.5 ,   3.75,
         4.  ], dtype=float32)

In [28]:
latitudes = uk_data.latitude.values
longitudes = uk_data.longitude.values
# print(latitudes.shape, longitudes.shape)

t_lat = torch.from_numpy(latitudes)
t_lon = torch.from_numpy(longitudes)

lat_lon_grid = torch.stack(
    torch.meshgrid(t_lat, t_lon, indexing="ij"), dim=-1
).permute(2, 1, 0)

lat_lon_grid.shape # (2, lon, lat) or (2, x, y)

grid_array = lat_lon_grid.numpy()
grid_array.shape
# np.save("nwp_xy.npy", grid_array)

(2, 57, 65)

In [64]:
np.load("data/era5_uk_reduced/static/nwp_xy.npy")

array([[[ 63.  ,  62.75,  62.5 , ...,  47.5 ,  47.25,  47.  ],
        [ 63.  ,  62.75,  62.5 , ...,  47.5 ,  47.25,  47.  ],
        [ 63.  ,  62.75,  62.5 , ...,  47.5 ,  47.25,  47.  ],
        ...,
        [ 63.  ,  62.75,  62.5 , ...,  47.5 ,  47.25,  47.  ],
        [ 63.  ,  62.75,  62.5 , ...,  47.5 ,  47.25,  47.  ],
        [ 63.  ,  62.75,  62.5 , ...,  47.5 ,  47.25,  47.  ]],

       [[-10.  , -10.  , -10.  , ..., -10.  , -10.  , -10.  ],
        [ -9.75,  -9.75,  -9.75, ...,  -9.75,  -9.75,  -9.75],
        [ -9.5 ,  -9.5 ,  -9.5 , ...,  -9.5 ,  -9.5 ,  -9.5 ],
        ...,
        [  3.5 ,   3.5 ,   3.5 , ...,   3.5 ,   3.5 ,   3.5 ],
        [  3.75,   3.75,   3.75, ...,   3.75,   3.75,   3.75],
        [  4.  ,   4.  ,   4.  , ...,   4.  ,   4.  ,   4.  ]]],
      dtype=float32)

In [ ]:
processed_data_path="data/era5_uk_reduced/samples/train"
sample_name="20220101000000"
sample_path=f"{processed_data_path}/{sample_name}.npy"

sample=np.load(sample_path)

In [ ]:
import torch

In [29]:
# lat, lon
input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
latitudes

tensor([-90.0000, -89.7500, -89.5000, -89.2500, -89.0000, -88.7500, -88.5000,
        -88.2500, -88.0000, -87.7500, -87.5000, -87.2500, -87.0000, -86.7500,
        -86.5000, -86.2500, -86.0000, -85.7500, -85.5000, -85.2500, -85.0000,
        -84.7500, -84.5000, -84.2500, -84.0000, -83.7500, -83.5000, -83.2500,
        -83.0000, -82.7500, -82.5000, -82.2500, -82.0000, -81.7500, -81.5000,
        -81.2500, -81.0000, -80.7500, -80.5000, -80.2500, -80.0000, -79.7500,
        -79.5000, -79.2500, -79.0000, -78.7500, -78.5000, -78.2500, -78.0000,
        -77.7500, -77.5000, -77.2500, -77.0000, -76.7500, -76.5000, -76.2500,
        -76.0000, -75.7500, -75.5000, -75.2500, -75.0000, -74.7500, -74.5000,
        -74.2500, -74.0000, -73.7500, -73.5000, -73.2500, -73.0000, -72.7500,
        -72.5000, -72.2500, -72.0000, -71.7500, -71.5000, -71.2500, -71.0000,
        -70.7500, -70.5000, -70.2500, -70.0000, -69.7500, -69.5000, -69.2500,
        -69.0000, -68.7500, -68.5000, -68.2500, -68.0000, -67.75

In [30]:
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
longitudes

tensor([-179.7500, -179.5000, -179.2500,  ...,  179.5000,  179.7500,
         180.0000])

In [31]:
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)

In [32]:
lat_lon_grid.shape

torch.Size([721, 1440, 2])

In [33]:
lat_lon_grid

tensor([[[ -90.0000, -179.7500],
         [ -90.0000, -179.5000],
         [ -90.0000, -179.2500],
         ...,
         [ -90.0000,  179.5000],
         [ -90.0000,  179.7500],
         [ -90.0000,  180.0000]],

        [[ -89.7500, -179.7500],
         [ -89.7500, -179.5000],
         [ -89.7500, -179.2500],
         ...,
         [ -89.7500,  179.5000],
         [ -89.7500,  179.7500],
         [ -89.7500,  180.0000]],

        [[ -89.5000, -179.7500],
         [ -89.5000, -179.5000],
         [ -89.5000, -179.2500],
         ...,
         [ -89.5000,  179.5000],
         [ -89.5000,  179.7500],
         [ -89.5000,  180.0000]],

        ...,

        [[  89.5000, -179.7500],
         [  89.5000, -179.5000],
         [  89.5000, -179.2500],
         ...,
         [  89.5000,  179.5000],
         [  89.5000,  179.7500],
         [  89.5000,  180.0000]],

        [[  89.7500, -179.7500],
         [  89.7500, -179.5000],
         [  89.7500, -179.2500],
         ...,
         [  89.75

In [39]:
torch.from_numpy(np.load("data/era5_global/static/nwp_xy.npy")).permute(2, 1, 0)

tensor([[[ 9.0000e+01,  0.0000e+00],
         [ 9.0000e+01,  2.5000e-01],
         [ 9.0000e+01,  5.0000e-01],
         ...,
         [ 9.0000e+01,  3.5925e+02],
         [ 9.0000e+01,  3.5950e+02],
         [ 9.0000e+01,  3.5975e+02]],

        [[ 8.9750e+01,  0.0000e+00],
         [ 8.9750e+01,  2.5000e-01],
         [ 8.9750e+01,  5.0000e-01],
         ...,
         [ 8.9750e+01,  3.5925e+02],
         [ 8.9750e+01,  3.5950e+02],
         [ 8.9750e+01,  3.5975e+02]],

        [[ 8.9500e+01,  0.0000e+00],
         [ 8.9500e+01,  2.5000e-01],
         [ 8.9500e+01,  5.0000e-01],
         ...,
         [ 8.9500e+01,  3.5925e+02],
         [ 8.9500e+01,  3.5950e+02],
         [ 8.9500e+01,  3.5975e+02]],

        ...,

        [[-8.9500e+01,  0.0000e+00],
         [-8.9500e+01,  2.5000e-01],
         [-8.9500e+01,  5.0000e-01],
         ...,
         [-8.9500e+01,  3.5925e+02],
         [-8.9500e+01,  3.5950e+02],
         [-8.9500e+01,  3.5975e+02]],

        [[-8.9750e+01,  0.0000e+00

In [ ]:
print(lat_lon_grid.shape)

In [ ]:
tmp = lat_lon_grid.permute(2, 0, 1)
tmp.shape

In [ ]:
tmp[1][0].shape

In [ ]:
tmp.view(2, -1).shape

In [ ]:
import json
import numpy as np

icospheres_path = "data/era5_global/icospheres.json"
with open(icospheres_path, "r") as f:
    loaded_dict = json.load(f)
    icospheres = {
        key: (np.array(value) if isinstance(value, list) else value)
        for key, value in loaded_dict.items()
    }

In [ ]:
for k in icospheres.keys():
    print(k)

In [ ]:
icospheres["order_0_face_centroid"]

In [ ]:
from graphcast_utils import latlon2xyz
from sklearn.neighbors import NearestNeighbors

input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)
lat_lon_grid_flat = lat_lon_grid.permute(2, 0, 1).view(2, -1).permute(1, 0)
lat_lon_grid_flat.shape

In [ ]:
cartesian_grid = latlon2xyz(lat_lon_grid_flat) # (lat*lon, 3)
n_nbrs = 4
neighbors = NearestNeighbors(n_neighbors=n_nbrs).fit(
    icospheres["order_" + str(6) + "_vertices"] # (40962, 3)
)

In [ ]:
distances, indices = neighbors.kneighbors(cartesian_grid) # (lat*lon, n_nbrs)

In [ ]:
lat_lon_grid_flat

In [ ]:
cartesian_grid

In [ ]:
src, dst = [], []
for i in range(len(cartesian_grid)):
    for j in range(n_nbrs):
        if distances[i][j] <= 0.6 * edge_len:
            src.append(i)
            dst.append(indices[i][j])

In [ ]:
np.load("data/era5_uk_reduced/samples/train/20220101000000.npy").shape # (6, 7, 65, 57) (levels, variables, lat, lon)
np.load("data/_era5_uk_reduced/samples/train/20220101000000.npy").shape # (6, 7, 721, 1440)

In [ ]:
uk_bbox = {
    "lat_max": 63,
    "lat_min": 47,
    "lon_max": 4,
    "lon_min": -10,
}

torch.linspace(uk_bbox["lat_min"], uk_bbox["lat_max"], steps=65) # (65,)

In [ ]:
torch.linspace(uk_bbox["lon_min"], uk_bbox["lon_max"], steps=57) # (65,)

In [65]:
import torch

# Assuming larger is your larger tensor and smaller is your smaller tensor
larger = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]])
smaller = torch.tensor([[1, 2], [7, 8]])

# Expand dimensions for broadcasting
larger_exp = larger.unsqueeze(1)
smaller_exp = smaller.unsqueeze(0)

# Compare all pairs of rows
matches = torch.all(larger_exp == smaller_exp, dim=-1)

# Get indices of matching rows
indices = torch.where(matches)

In [66]:
indices

(tensor([0, 3]), tensor([0, 1]))

In [100]:
import torch

# Example tensors
larger_tensor = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
smaller_tensor = torch.tensor([[5.0, 6.0], [3.0, 4.0], [1.0, 2.0]])

def find_subset_indices(larger, smaller):
    """
    finds the indices of the smaller tensor in the larger tensor
    """
    indices = []
    for i in range(smaller.size(0)):
        index = torch.where(torch.all(larger == smaller[i], dim=1))[0]
        if index.size(0) == 0:
            raise ValueError("The smaller tensor is not a subset of the larger tensor")
        indices.append(index.item())
    return indices

indices = find_subset_indices(larger_tensor, smaller_tensor)
print(indices)

[2, 1, 0]


In [103]:
import torch
import numpy as np

# Assuming tensor1 and tensor2 are your tensors
tensor1 = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]])
tensor2 = torch.tensor([[1, 2], [7, 8], [9, 10]])

# Convert tensors to numpy arrays
array1 = tensor1.numpy()
array2 = tensor2.numpy()

# Find common rows
common_rows = np.intersect1d(array1, array2, axis=0)

# Convert back to tensor
common_rows_tensor = torch.from_numpy(common_rows)

TypeError: intersect1d() got an unexpected keyword argument 'axis'

In [104]:
a = list(range(10))
b = [3, 2 , 1]

a[b]

TypeError: list indices must be integers or slices, not list

In [1]:
w_dict = {
    "2": 1.0,
    "0": 0.1,
    "65": 0.065,
    "1000": 0.1,
    "850": 0.05,
    "500": 0.03,
}
w_list = np.array(
    [w_dict[par.split("_")[-2]] for par in constants.PARAM_NAMES]
)

NameError: name 'np' is not defined